<a href="https://colab.research.google.com/github/SisekoC/My-Notebooks/blob/main/resnet_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import  Activation , Conv2D , BatchNormalization , Add , MaxPool2D , GlobalAveragePooling2D , Dense
import tensorflow_datasets as tfds
print("Version of tensorflow is : ",tf.__version__)
from tensorflow.keras.utils import plot_model

Version of tensorflow is :  2.17.0


In [2]:
class IdentityBlock(Model):
    def __init__(self , filters , kernal_size):
        super(IdentityBlock , self ).__init__(name  ='')
        self.conv = Conv2D(filters , kernal_size , padding='same')
        self.norm = BatchNormalization()
        self.act  = Activation('relu')
        self.add  = Add()
    def call(self , input):
        x = self.conv(input)
        x = self.norm(x)
        x = self.act(x)

        x = self.conv(x)
        x = self.norm(x)

        x = self.add([x , input])
        x = self.act(x)
        return x

In [3]:
class Resnet(Model):
    def __init__(self , num_classes , activation = 'softmax'):
        super(Resnet , self).__init__(name="")
        self.conv7 = Conv2D(64 , 7 , padding='same')
        self.norm  = BatchNormalization()
        self.pool  = MaxPool2D((3,3))
        self.idbl1 = IdentityBlock(64 , 3)
        self.idbl2 = IdentityBlock(64 , 3)
        self.gpool = GlobalAveragePooling2D()
        self.claasifier = Dense(num_classes , activation = tf.keras.activations.get(activation))

    def call(self , input):
        x = self.conv7(input)
        x = self.norm(x)
        x = self.pool(x)
        x = self.idbl1(x)
        x = self.idbl2(x)
        x = self.gpool(x)
        x = self.claasifier(x)

        return x

In [4]:
resnet_model = Resnet(10)
resnet_model.compile(optimizer='adam' , loss = 'sparse_categorical_crossentropy' , metrics=['accuracy'])

In [5]:
def preprocess(f):
    return tf.cast(f['image'] , 'float32') / 255.0  , f["label"]

dataset = tfds.load(name='mnist' , split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
history = resnet_model.fit(dataset , epochs=3)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 240s 126ms/step - accuracy: 0.7577 - loss: 0.6795
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 229s 122ms/step - accuracy: 0.9725 - loss: 0.0905
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 227s 121ms/step - accuracy: 0.9821 - loss: 0.0584


In [7]:
dataset_test = tfds.load(name='mnist' , split=tfds.Split.TEST)
dataset_test = dataset_test.map(preprocess).batch(32)

In [8]:
resnet_model.evaluate(dataset_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.9871 - loss: 0.0444


[0.04840370640158653, 0.9853000044822693]

In [9]:
resnet_model.summary()

Model: ""

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 64)          │           3,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 9, 9, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│  (IdentityBlock)                     │ ?                           │          37,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│  (IdentityBlock)                     │ ?                           │          37,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 234,656 (916.63 KB)

 Trainable params: 78,090 (305.04 KB)

 Non-trainable params: 384 (1.50 KB)

 Optimizer params: 156,182 (610.09 KB)